In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import numpy as np
import lxml 

In [17]:

# https://www.perfectgame.org/Rankings/Players/NationalRankings.aspx
# params: gyear 
# https://www.perfectgame.org/Rankings/Players/FollowList.aspx
# params: gyear
# https://www.perfectgame.org/Rankings/Players/HighFollowList.aspx
# params: gyear

# https://www.baseballfactory.com/recruiting-and-promotion/the-top-100/?y=2019
# https://athlonsports.com/mlb/2021-mlb-draft-top-50-high-school-draft-prospects
urls = {'perfect_game':'https://www.perfectgame.org/Rankings/Players/NationalRankings.aspx',
        'baseball_factory':'https://www.baseballfactory.com/recruiting-and-promotion/the-top-100/', 
       }

In [18]:
source = 'perfect_game'
year = 2017
payload = {'gyear':str(year)}
url = urls[source]
r = requests.get(url, params=payload)
response = r.text
soup = BeautifulSoup(response)
table = soup.find(id='ContentTopLevel_ContentPlaceHolder1_gvPlayers')

# a IDs
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_hlPlayerName_0
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_hl4yr_0
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_hlTournamentTeam_0

# span IDs
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblNatRank_
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblHeight_
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblWeight_
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblBatsThrows_
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblHometown_
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblDrafted_
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblDebuted_
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblTravelTeam_
# ContentTopLevel_ContentPlaceHolder1_gvPlayers_InternalCommentLiteral_

v = table.find(id='ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblNatRank_0').string
str(v)

'1'

In [59]:
ids = {'player_name':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_hlPlayerName_',
    'college':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_hl4yr_',
    'rank':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblNatRank_',
    'height':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblHeight_',
    'weight':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblWeight_',
    'bats/throws':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblBatsThrows_',
    'hometown':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblHometown_',
    'drafted':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblDrafted_',
    'debuted':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblDebuted_',
    'travel_team':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_lblTravelTeam_',
    'notes':'ContentTopLevel_ContentPlaceHolder1_gvPlayers_InternalCommentLiteral_'
      }
source = 'perfect_game'
url = urls[source]

dfs = {}
for year in range(2002, 2022):
    payload = {'gyear':str(year)}
    r = requests.get(url, params=payload)
    response = r.text
    soup = BeautifulSoup(response)
    table = soup.find(id='ContentTopLevel_ContentPlaceHolder1_gvPlayers')

    
    data = {}
    for i in range(100):
        details = []
        for key in ids:
            try: 
                details.append(str(table.find(id=ids[key]+str(i)).string))
            except: 
                details.append('NULL')
        data[i]=details
    df = pd.DataFrame.from_dict(data, orient='index')
    df.columns = ids.keys()
    df = df.iloc[:50]
    df['class_year'] = year
    dfs[str(year)] = df
    

In [80]:
data = pd.DataFrame()
for i in dfs.items():
    data =data.append(i[1])
data.to_csv('data/perfectgame/perfectgame_top50_2002-2021_raw.csv', index=False)

In [ ]:
source = 'baseball_factory'
year = 2017
payload = {'y':str(year)}
url = urls[source]
r = requests.get(url, params=payload)
response = r.text
soup = BeautifulSoup(response)
a = soup.find_all('article', id='prospects')[0]
# class="rank"

In [ ]:
res = []
last = ''
k = 1
for i in a.descendants: 
    if i.string is not None: 
        if i.string != '\n':
            if i.string != 'Watch Video':
                if i.string != last: 
                    if k % 22 == 0: 
                        details = ''
                        details+= (str(i.string))+', '
                    if k % 21 == 0:
                        res.append((k, details))
                    last = i.string
                    k+=1 
res

